In [1]:
from utils import *

In [2]:
from scipy.stats import pearsonr
import seaborn as sns
from IPython.display import Markdown

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
bac = pd.read_excel("data/bac_genus.xlsx", sheet_name=None)

In [5]:
bac['m1_rep1'].keys()

Index(['Genus', 'd0con', 'd7con', 'd15con', 'd7ab', 'd15dss_ab', 'd7dss',
       'd15ab_dss', 'd15dss_h2o', 'd15ab_h2o', 'd7dab', 'd15dab_h2o'],
      dtype='object')

In [6]:
points_bac = ['d0con', 'd7con', 'd7dss', 'd7ab', 'd7dab', 'd15con', 'd15dss_ab', 'd15ab_dss', 'd15dss_h2o', 'd15ab_h2o', 'd15dab_h2o']

In [7]:
genus_list = np.array(bac['m1_rep1']['Genus'])

In [8]:
points_mb = ['0_con', '7_con', '7_dss', '7_ab', '7_dab', '15_con', '15_dss_ab', '15_ab_dss', '15_dss_h2o', '15_ab_h2o', '15_dab_h2o']

### elimination procedure

for metabolites we consider cvfiltered list

for bacteria data the selecton ids based on where it has two or more non zero values corresponding to 3 replicated.

#### m1

In [9]:
! mkdir -p generated/pointcorr/m1
! mkdir -p generated/pointcorr/m1/raw

In [10]:
mb_m1 = list(pd.read_hdf('generated/a_cecal_mouse_1.hdf5', 'cvfiltered')['Metabolite Name'])
for i, _ in enumerate(points_bac):
    mb_rep1 = pd.read_hdf('generated/a_cecal_mouse_1.hdf5', 'rep_0')[['Metabolite Name', points_mb[i]]]
    mb_rep2 = pd.read_hdf('generated/a_cecal_mouse_1.hdf5', 'rep_1')[['Metabolite Name', points_mb[i]]]
    mb_rep3 = pd.read_hdf('generated/a_cecal_mouse_1.hdf5', 'rep_2')[['Metabolite Name', points_mb[i]]]

    mb_rep1 = mb_rep1.rename(columns={points_mb[i] : 'rep1'})
    mb_rep2 = mb_rep2.rename(columns={points_mb[i] : 'rep2'})
    mb_rep3 = mb_rep3.rename(columns={points_mb[i] : 'rep3'})

    merged_mb = mb_rep1.merge(mb_rep2, how='inner', on='Metabolite Name')
    merged_mb = merged_mb.merge(mb_rep3, how='inner', on='Metabolite Name')

    merged_mb = merged_mb.set_index('Metabolite Name')
    merged_mb = merged_mb[['rep1', 'rep2', 'rep3']]


    bac_rep1 = bac['m1_rep1'][['Genus', points_bac[i]]]
    bac_rep2 = bac['m1_rep2'][['Genus', points_bac[i]]]
    bac_rep3 = bac['m1_rep3'][['Genus', points_bac[i]]]

    bac_rep1 = bac_rep1.rename(columns={points_bac[i] : 'rep1'})
    bac_rep2 = bac_rep2.rename(columns={points_bac[i] : 'rep2'})
    bac_rep3 = bac_rep3.rename(columns={points_bac[i] : 'rep3'})

    merged_bac = bac_rep1.merge(bac_rep2, how='inner', on='Genus')
    merged_bac = merged_bac.merge(bac_rep3, how='inner', on='Genus')

    merged_bac = merged_bac.set_index('Genus')
    merged_bac = merged_bac[['rep1', 'rep2', 'rep3']]

    mask1 = (merged_bac['rep1'] != 0)
    mask2 = (merged_bac['rep2'] != 0)
    mask3 = (merged_bac['rep3'] != 0)

    maskall3 = mask1 & mask2 & mask3
    maskall2 = (mask1 & mask2 & ~mask3) | (mask1 & ~mask2 & mask3) | (~mask1 & mask2 & mask3)

    merged_bac = merged_bac[maskall3 | maskall2]

    smatrix = np.zeros([len(merged_bac), len(merged_mb)])
    pmatrix = np.zeros([len(merged_bac), len(merged_mb)])
    for b in range(len(merged_bac)):
        for j in range(len(merged_mb)):
            s, p = pearsonr(merged_mb.iloc[j].to_numpy(), merged_bac.iloc[b].to_numpy())
            smatrix[b, j] = s
            pmatrix[b, j] = p
        print(merged_bac.index[b], end=' ')
    corr_df = pd.DataFrame(smatrix, columns=merged_mb.index, index=merged_bac.index)
    smatrix_sig = smatrix.copy()

    np.save(f"generated/pointcorr/m1/raw/raw_S_genus_{points_bac[i]}.npy", smatrix)
    np.save(f"generated/pointcorr/m1/raw/raw_P_genus_{points_bac[i]}.npy", pmatrix)
    
    maskpm = pmatrix > 0.15      #    pearson p
    smatrix_sig[maskpm] = 0
    corr_df_sig = pd.DataFrame(smatrix_sig, columns=merged_mb.index, index=merged_bac.index)
    corr_df_sig = corr_df_sig.fillna(0)
    corr_df_sig = corr_df_sig.loc[(corr_df_sig != 0).any(axis=1)]
    corr_df_sig = corr_df_sig.loc[:, (corr_df_sig != 0).any(axis=0)]
    corr_df_sig[corr_df_sig == 0] = np.nan
    print(corr_df_sig.shape)

    corr_df_sig.to_csv(f"generated/pointcorr/m1/genus_{points_bac[i]}.csv")
    
    # pyplot.rcParams['figure.figsize'] = (25, 15)
    # try:
    #     sns.heatmap(corr_df_sig, cmap='coolwarm')
    #     pyplot.title(f'm1_genus_mb_{points_bac[i]}')
    #     pyplot.show()
    # except:
    #     print('can\'t plot', points_bac[i])
    #     pyplot.cla()

Alistipes Anaerotruncus Bacteroides Blautia Clostridium Desulfovibrio Eubacterium Gastranaerophilales Helicobacter Incertae_Sedis Lachnospiraceae Marvinbryantia Mucispirillum Muribaculaceae Prevotellaceae Rikenella Ruminococcus Sphingomonas UBA1819 UCG-009 (20, 16028)
A2 Acetatifactor Anaerotruncus Bacteroides Blautia Clostridium Desulfovibrio Enterorhabdus Eubacterium GCA-900066575 Helicobacter Incertae_Sedis Lachnoclostridium Lachnospiraceae Marvinbryantia Muribaculaceae Oscillibacter Roseburia Ruminococcus Sphingomonas UBA1819 (21, 15390)
Anaerotruncus Bacteroides Blautia Clostridium Eubacterium Helicobacter Incertae_Sedis Lachnospiraceae Muribaculaceae Ruminococcus Sphingomonas UBA1819 (12, 16150)
Ochrobactrum Sphingomonas (2, 1648)
Akkermansia Anaerosporobacter_mobilis Anaerotruncus Blautia Clostridium Enterobacter Erysipelatoclostridium Eubacterium Intestinimonas Lachnoclostridium Lachnotalea Muribaculaceae Robinsoniella Ruminococcus Sphingomonas Turicibacter (16, 16347)
A2 Aceta

#### m2

In [11]:
! mkdir -p generated/pointcorr/m2
! mkdir -p generated/pointcorr/m2/raw

In [12]:
mb_m2 = list(pd.read_hdf('generated/a_cecal_mouse_2.hdf5', 'cvfiltered')['Metabolite Name'])
for i, _ in enumerate(points_bac):
    mb_rep1 = pd.read_hdf('generated/a_cecal_mouse_2.hdf5', 'rep_0')[['Metabolite Name', points_mb[i]]]
    mb_rep2 = pd.read_hdf('generated/a_cecal_mouse_2.hdf5', 'rep_1')[['Metabolite Name', points_mb[i]]]
    mb_rep3 = pd.read_hdf('generated/a_cecal_mouse_2.hdf5', 'rep_2')[['Metabolite Name', points_mb[i]]]

    mb_rep1 = mb_rep1.rename(columns={points_mb[i] : 'rep1'})
    mb_rep2 = mb_rep2.rename(columns={points_mb[i] : 'rep2'})
    mb_rep3 = mb_rep3.rename(columns={points_mb[i] : 'rep3'})

    merged_mb = mb_rep1.merge(mb_rep2, how='inner', on='Metabolite Name')
    merged_mb = merged_mb.merge(mb_rep3, how='inner', on='Metabolite Name')

    merged_mb = merged_mb.set_index('Metabolite Name')
    merged_mb = merged_mb[['rep1', 'rep2', 'rep3']]


    bac_rep1 = bac['m2_rep1'][['Genus', points_bac[i]]]
    bac_rep2 = bac['m2_rep2'][['Genus', points_bac[i]]]
    bac_rep3 = bac['m2_rep3'][['Genus', points_bac[i]]]

    bac_rep1 = bac_rep1.rename(columns={points_bac[i] : 'rep1'})
    bac_rep2 = bac_rep2.rename(columns={points_bac[i] : 'rep2'})
    bac_rep3 = bac_rep3.rename(columns={points_bac[i] : 'rep3'})

    merged_bac = bac_rep1.merge(bac_rep2, how='inner', on='Genus')
    merged_bac = merged_bac.merge(bac_rep3, how='inner', on='Genus')

    merged_bac = merged_bac.set_index('Genus')
    merged_bac = merged_bac[['rep1', 'rep2', 'rep3']]

    mask1 = (merged_bac['rep1'] != 0)
    mask2 = (merged_bac['rep2'] != 0)
    mask3 = (merged_bac['rep3'] != 0)

    maskall3 = mask1 & mask2 & mask3
    maskall2 = (mask1 & mask2 & ~mask3) | (mask1 & ~mask2 & mask3) | (~mask1 & mask2 & mask3)

    merged_bac = merged_bac[maskall3 | maskall2]

    smatrix = np.zeros([len(merged_bac), len(merged_mb)])
    pmatrix = np.zeros([len(merged_bac), len(merged_mb)])
    for b in range(len(merged_bac)):
        for j in range(len(merged_mb)):
            s, p = pearsonr(merged_mb.iloc[j].to_numpy(), merged_bac.iloc[b].to_numpy())
            smatrix[b, j] = s
            pmatrix[b, j] = p
        print(merged_bac.index[b], end=' ')
    corr_df = pd.DataFrame(smatrix, columns=merged_mb.index, index=merged_bac.index)
    smatrix_sig = smatrix.copy()

    np.save(f"generated/pointcorr/m2/raw/raw_S_genus_{points_bac[i]}.npy", smatrix)
    np.save(f"generated/pointcorr/m2/raw/raw_P_genus_{points_bac[i]}.npy", pmatrix)
    
    maskpm = pmatrix > 0.15      #    pearson p
    smatrix_sig[maskpm] = 0
    corr_df_sig = pd.DataFrame(smatrix_sig, columns=merged_mb.index, index=merged_bac.index)
    corr_df_sig = corr_df_sig.fillna(0)
    corr_df_sig = corr_df_sig.loc[(corr_df_sig != 0).any(axis=1)]
    corr_df_sig = corr_df_sig.loc[:, (corr_df_sig != 0).any(axis=0)]
    corr_df_sig[corr_df_sig == 0] = np.nan
    print(corr_df_sig.shape)

    corr_df_sig.to_csv(f"generated/pointcorr/m2/genus_{points_bac[i]}.csv")
    
    # pyplot.rcParams['figure.figsize'] = (25, 15)
    # try:
    #     sns.heatmap(corr_df_sig, cmap='coolwarm')
    #     pyplot.title(f'm2_genus_mb_{points_bac[i]}')
    #     pyplot.show()
    # except:
    #     print('can\'t plot', points_bac[i])
    #     pyplot.cla()

A2 Acetatifactor Anaeroplasma Bacteroides Blautia Candidatus_Saccharimonas Clostridium Eubacterium GCA-900066575 Helicobacter Incertae_Sedis Lachnoclostridium Lachnospiraceae Muribaculaceae Odoribacter Oscillibacter Parabacteroides Prevotellaceae Rikenella Roseburia Ruminococcus Sphingomonas (22, 16791)
A2 Acetatifactor Bacteroides Clostridium Desulfovibrio Eubacterium F082 GCA-900066575 Helicobacter Incertae_Sedis Lachnoclostridium Lachnospiraceae Muribaculaceae Odoribacter Oscillibacter Prevotellaceae Rikenella Roseburia Ruminococcus Sphingomonas (20, 16331)
Anaerotruncus Bacteroides GCA-900066575 Helicobacter Incertae_Sedis Lachnospiraceae Muribaculaceae Ruminococcus Sphingomonas (9, 12936)
Bacteroides Helicobacter Sphingomonas (3, 2598)
Bacteroides Blautia Clostridium Erysipelatoclostridium Eubacterium Lachnoclostridium Lachnospiraceae Marvinbryantia Ruminococcus Sphingomonas (10, 15412)
A2 Acetatifactor Alistipes Anaerotruncus Bacteroides Blautia Clostridium Desulfovibrio Enterorh